# Playing Frozen Lake with Q-Learning

In this tutorial, we are going to be exploring a family of RL algorithms called Q-Learning algorithms. These are a little different than the policy-based algorithms. Instead of starting with a complex and unwieldy deep neural network, we will begin by implementing a simple lookup-table version of the algorithm.

Let’s start building our Q-table algorithm, which will try to solve Frozen Lake environment. In this environment aim is to reach the goal, on a frozen lake that might have some holes in it. Here is how surface is depicted by this algorithm.



    SFFF       (S: starting point, safe) 
    FHFH       (F: frozen surface, safe)
    FFFH       (H: hole, fall to your doom)
    HFFG       (G: goal, where the frisbee is located)

### 1- Install gym library

In [ ]:
!pip install gym

### 2- Import libraries

In [ ]:
import numpy as np
import gym
import random
import time
from IPython.display import clear_output

### 3- Load Gym environment for Frozen Lake game

In [ ]:
env = gym.make("FrozenLake-v0")
action_space_size = env.action_space.n
state_space_size = env.observation_space.n

### 4- Initialize Q-table

**<span style="color:blue">Initializing a q-table means that we need to create a 2D matrix (array), set the right dimensions and all cells set to 0. This can be done using "numpy" library- specifically lookup nump.zeros() and you will be on your way for solving it! Q table needs to have its number of rows equal to "state_space_size" parameter and its columns need to be equal to "action_space_size".</span>**

In [ ]:
######################################################################
#               Your code for initializing q table goes here
q_table = np.zeros((state_space_size, action_space_size))
######################################################################

print(q_table)

### 5- Set Q-learning parameters

**<span style="color:blue">"num_episodes" is the maximum number of times (episodes) we want to play the game to learn it while training. There needs to be enough number of times so that the agent has enough time to learn how to play. For this example one hundred thousands episodes is required to make sure the agent can learn how to play the game. </span>**
    
**<span style="color:blue">"max_steps_per_episode" is the number steps the agent can take each time it plays the game, to get to the goal, without failing. We should not choose a very large value for this because we want the agent to learn to get to the goal quickly. For this example, we can try a few hundreds of steps per episode maximum. </span>**

**<span style="color:blue">"learning_rate", is a parameter that defines how fast we want to update the Q-table. It is usually a small number at the lower range of [0, 1] interval. The lower the learning rate is, the slower Q-table will be updated and vice versa. </span>**

In [ ]:
num_episodes = 100000
max_steps_per_episode = 100

learning_rate = 0.01
discount_rate = 0.99

exploration_rate = .1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.01
rewards_all_episodes = []

### 6- Bellman equation

$$Q^{New}_{(s,a)} = Q_{(s,a)} + \alpha [\Delta Q_{(s,a)}]$$

where:

$$\Delta Q_{(s,a)} =  R_{(s,a)} + \gamma\max_a Q'_{(s',a')} - Q_{(s,a)}$$


Bellman equation can also be written in the following form:

$$Q^{New}_{(s,a)} = Q_{(s,a)}(1 - \alpha) + \alpha [R_{(s,a)} + \gamma\max_a Q'_{(s',a')}]$$

where:
    
$$Q^{New}_{(s,a)} : new \ Q \ value$$
$$Q_{(s,a)} : current \ Q \ value$$   
$$ \alpha : learning \ rate$$ 
$$R_{(s,a)} : reward \ for \  taking \  action\  a \ at\  state\  s $$
$$\gamma : discount \ rate $$
$$\max_a Q'_{(s',a')} :  maximum \ expected \ future \ reward$$

In [ ]:
def Bellman(q_table1, action1, state1, learning_rate1, reward1, discount_rate1,new_state1):
    
    Q_sa_new = q_table1[state1, action1] * (1 - learning_rate1) + learning_rate1 *\
    (reward1 + discount_rate1 * np.max(q_table1[new_state1, :]))
    return Q_sa_new

### 7- Exploration decay equation

$$ \epsilon  = \epsilon_{min} + (\epsilon_{max} - \epsilon_{min}) \exp(- d_{\epsilon} * episode)$$

where:


$$\epsilon : exploration \ rate$$ 
$$\epsilon_{min} : min \ exploration \ rate$$ 
$$\epsilon_{max} : max \ exploration \ rate$$ 
$$d_{\epsilon} : exploration\ decay \ rate $$
$$episode : episode \ number$$

In [ ]:
def Exploration_decay(min_exploration_rate1, max_exploration_rate1, exploration_decay_rate1, episode1):
    
    exploration_rate1 = min_exploration_rate1 + \
    (max_exploration_rate1 - min_exploration_rate1) * np.exp(-exploration_decay_rate1*episode1)
    return exploration_rate1

### 8- Train/ Update Q table

In [ ]:
for episode in range(num_episodes):
    state = env.reset()
    done = False
    rewards_current_episode = 0

    for step in range(max_steps_per_episode): 
        # Exploration-exploitation trade-off
        exploration_rate_threshold = random.uniform(0, 1)
        if exploration_rate_threshold > exploration_rate:
            action = np.argmax(q_table[state,:]) 
        else:
            action = env.action_space.sample()
        new_state, reward, done, info = env.step(action)
    
        # Update Q-table for Q(s,a)
        qsa = Bellman(q_table, action, state, learning_rate, reward, discount_rate,new_state)
        q_table[state, action] = qsa
        state = new_state
        rewards_current_episode += reward 
        if done == True: 
            break
    # Exploration rate decay
    exploration_rate = Exploration_decay(min_exploration_rate, max_exploration_rate, exploration_decay_rate, episode)
    rewards_all_episodes.append(rewards_current_episode)

### 9- Calculate and print the average reward per thousand episodes

In [ ]:
rewards_per_thosand_episodes = np.split(np.array(rewards_all_episodes),num_episodes/1000)
count = 1000

print("********Average reward per thousand episodes********\n")
for r in rewards_per_thosand_episodes:
    print(count, ": ", str(sum(r/1000)))
    count += 1000

### 10- Check your updated Q table

In [ ]:
q_table

### 11- Let it play the game!

In [ ]:
for episode in range(10):
    state = env.reset()
    done = False
    print("*****EPISODE ", episode+1, "*****\n\n\n\n")
    time.sleep(1)
    for step in range(max_steps_per_episode):        
        clear_output(wait=True)
        env.render()
        time.sleep(0.3)
        
        action = np.argmax(q_table[state,:])        
        new_state, reward, done, info = env.step(action)
        
        if done:
            clear_output(wait=True)
            env.render()
            if reward == 1:
                print("****You reached the goal!****")
                time.sleep(3)
            else:
                print("****You fell through a hole!****")
                time.sleep(3)
                clear_output(wait=True)
            break
            
        state = new_state
        env.close()
